In [1]:
import pandas as pd
import numpy as np

from data_processing import *
from utils import feature_desc

from sklearn.preprocessing import StandardScaler

#### Load data

In [2]:
df_og = load(csv_path='../data/merged.csv')

#### Aggregate time into clusters of size k

In [3]:
k = 15
df_og = aggregate(df_og, k)

### Proste featury wybieram albo tworze agregując


In [4]:
df = pd.DataFrame(index=df_og.index)

In [10]:
# srednia po 'WODY POWROTNE KOLEKTORÓW [°C]'
TIR = df_og.columns[df_og.columns.str.contains('tir')].values
df['TIR'] = df_og[TIR].mean(axis='columns')

In [11]:
# srednia po 'TEMP POD 2 WARSTWĄ WYMURÓWKI [°C]'
TIX1 = df_og.columns[df_og.columns.str.contains('001tix')].values
df['TIX1'] = df_og[TIX1].mean(axis='columns')

In [12]:
# prob_s i prob_corg
PR = ['prob_s', 'prob_corg']
df[PR] = df_og[PR]

In [16]:
# reg nadawy koncentratu
FCX = df_og.columns[df_og.columns.str.contains('fcx')].values
df[FCX] = df_og[FCX]

In [17]:
# sumaryczna moc cieplna
NIR = df_og.columns[df_og.columns.str.contains('nir')].values
df[NIR] = df_og[NIR]

In [18]:
# WENT ODCZ ZAD OBROTÓW
UXM = df_og.columns[df_og.columns.str.contains('uxm')].values
df['UXM'] = df_og[UXM].mean(axis='columns')

In [19]:
# dodanie temperatury żużla
df["TEMP_ZUZ"] = df_og["temp_zuz"]

In [20]:
df

,TIR,TIX1,prob_s,prob_corg,UXM,001fcx00211.pv,001fcx00221.pv,001fcx00231.pv,001fcx00241.pv,001nir0szr0.daca.pv,TEMP_ZUZ
czas,,,,,,,,,,,
2020-09-30 22:00:00+00:00,29.086390,418.990965,9.87,8.6,92.173271,56.729077,54.724422,11.966905,22.493207,14.362428,1297.0
2020-09-30 22:01:00+00:00,29.087996,418.990059,9.87,8.6,92.173326,54.771942,54.734675,12.026410,22.593412,14.381825,NaN
2020-09-30 22:02:00+00:00,29.094825,418.989153,9.87,8.6,92.173381,54.695816,54.774163,11.953671,22.428933,14.359413,NaN
2020-09-30 22:03:00+00:00,29.098936,418.988247,9.87,8.6,92.173436,54.154394,54.985713,12.052065,22.335388,14.353036,NaN
2020-09-30 22:04:00+00:00,29.096812,418.987341,9.87,8.6,92.173490,54.693184,54.490742,12.027310,22.412620,14.392053,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2021-10-11 06:07:00+00:00,28.166006,417.488689,11.02,8.4,97.294037,58.795139,54.973000,10.087992,27.894094,14.648799,NaN
2021-10-11 06:08:00+00:00,28.173012,417.488600,11.02,8.4,97.291126,54.448054,54.905028,9.947703,28.020189,14.637558,NaN
2021-10-11 06:09:00+00:00,28.171779,417.488511,11.02,8.4,97.293714,53.660009,55.608914,9.886163,28.001451,14.623958,NaN


## Data is now ready to do somthing with it idk